In [2]:
!apt install lynx

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  lynx-common mailcap mime-support
The following NEW packages will be installed:
  lynx lynx-common mailcap mime-support
0 upgraded, 4 newly installed, 0 to remove and 33 not upgraded.
Need to get 1,771 kB of archives.
After this operation, 5,706 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 lynx-common all 2.9.0dev.10-1 [1,024 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 lynx amd64 2.9.0dev.10-1 [719 kB]
Fetched 1,771 kB in 1s (1,762 kB/s)
Selecting previously unselected package lynx-common.
(Reading database ... 121749 files and directories currently installed.)
Preparing to unpack .

In [ ]:
!lynx -dump https://librivox.org/dubliners-by-james-joyce/|grep mp3|awk '{print $NF}'|grep 64kb|grep 'mp3$' > dubliners1.txt
!lynx -dump https://librivox.org/dubliners-by-james-joyce-2/|grep mp3|awk '{print $NF}'|grep 64kb|grep 'mp3$' > dubliners2.txt
!wget -i dubliners1.txt
!wget -i dubliners2.txt

--2024-02-15 16:27:29--  https://www.archive.org/download/dubliners_0906_librivox/dubliners_01_joyce_64kb.mp3
Resolving www.archive.org (www.archive.org)... 207.241.224.2
Connecting to www.archive.org (www.archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://archive.org/download/dubliners_0906_librivox/dubliners_01_joyce_64kb.mp3 [following]
--2024-02-15 16:27:30--  https://archive.org/download/dubliners_0906_librivox/dubliners_01_joyce_64kb.mp3
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia902800.us.archive.org/3/items/dubliners_0906_librivox/dubliners_01_joyce_64kb.mp3 [following]
--2024-02-15 16:27:30--  https://ia902800.us.archive.org/3/items/dubliners_0906_librivox/dubliners_01_joyce_64kb.mp3
Resolving ia902800.us.archive.org (ia902800.us.archive.org)... 207.241

In [ ]:
!for i in *.mp3.1;do mv $i $(basename $i .mp3.1)_2.mp3;done

Here starts the actual ASR stuff.

In [ ]:
!pip install transformers

In [ ]:
EN_MODEL = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
PHONE_MODEL = "vitouphy/wav2vec2-xls-r-300m-timit-phoneme"

In [ ]:
from transformers import pipeline

In [ ]:
pipe_en = pipeline(model=EN_MODEL)
pipe_phone = pipeline(model=PHONE_MODEL)

In [ ]:
from pathlib import Path
import json
cur = Path(".")
for file in cur.glob("*.mp3"):
    en_out = pipe_en(str(file), chunk_length_s=10, return_timestamps="word")
    phone_out = pipe_phone(str(file), chunk_length_s=10, return_timestamps="word")
    stem = file.stem
    with open(f"{stem}_en.json") as enfile:
        json.dump(en_out, enfile)
    with open(f"{stem}_phone.json") as phonefile:
        json.dump(phone_out, phonefile)